In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [5]:
# Load API Credentials
with open('/Users/christinab/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

### Define Search

In [30]:
# set API call parameters 
location = 'New York, NY 10002'
term = 'Indian'

In [31]:
# create the JSON data folder 
JSON_FILE = "Data/results_in_progress_NY_Indian.json"
JSON_FILE

'Data/results_in_progress_NY_Indian.json'

In [32]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_Indian.json already exists.


In [33]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [34]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location='New York, NY 10002',
                                term='Indian',
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [35]:
## How many results total?
total_results = results['total']
total_results


110

In [36]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [37]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

6

In [38]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [40]:
# set up a progress bar for your loop 
!pip install tqdm

In [41]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/6 [00:00<?, ?it/s]

In [43]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location='New York, NY 10002',
                                    term='Indian', 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/6 [00:00<?, ?it/s]

In [44]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Txm8KtUh9Mn2E7980jTr5g,dhamaka-new-york,Dhamaka,https://s3-media4.fl.yelpcdn.com/bphoto/O7stEz...,False,https://www.yelp.com/biz/dhamaka-new-york?adju...,293,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 40.7180658, 'longitude': -73.9881...",[delivery],$$$,"{'address1': '119 Delancey St', 'address2': 'E...",+12122048616,(212) 204-8616,271.065740
1,_7BGw3YFNOTzP1Www3zB7g,mughlai-grill-new-york,Mughlai Grill,https://s3-media3.fl.yelpcdn.com/bphoto/R0MIGO...,False,https://www.yelp.com/biz/mughlai-grill-new-yor...,471,"[{'alias': 'indpak', 'title': 'Indian'}]",4.5,"{'latitude': 40.7210905, 'longitude': -73.9840...","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '6 Clinton St', 'address2': '', '...",+12125332828,(212) 533-2828,512.531279
2,Xqk91WuhMeYS4pedccUYgA,gazab-new-york,Gazab,https://s3-media3.fl.yelpcdn.com/bphoto/AuOMEQ...,False,https://www.yelp.com/biz/gazab-new-york?adjust...,157,"[{'alias': 'indpak', 'title': 'Indian'}]",4.5,"{'latitude': 40.72191, 'longitude': -73.98666}","[delivery, pickup]",NaN,"{'address1': '179 Essex St', 'address2': '', '...",+19173882664,(917) 388-2664,597.003167
3,zyQpS4QrYYB0VKXf7JPUlQ,atithi-indian-cuisine-brooklyn,Atithi Indian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/1HW_F-...,False,https://www.yelp.com/biz/atithi-indian-cuisine...,146,"[{'alias': 'indpak', 'title': 'Indian'}]",5.0,"{'latitude': 40.714765, 'longitude': -73.961606}","[delivery, pickup, restaurant_reservation]",$$,"{'address1': '159 Grand St', 'address2': None,...",+17186849192,(718) 684-9192,2029.536488
4,7DUPGtHqVZHJ3yIufbWLsg,malai-marke-indian-cuisine-new-york,Malai Marke Indian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/jKKQcL...,False,https://www.yelp.com/biz/malai-marke-indian-cu...,631,"[{'alias': 'indpak', 'title': 'Indian'}]",4.0,"{'latitude': 40.726934, 'longitude': -73.98758}","[delivery, pickup]",$$,"{'address1': '318 E 6th St', 'address2': None,...",+12127777729,(212) 777-7729,1159.430082


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
105,WBJLoLeRSSzRDrzOv5IRng,sonali-indian-restaurant-new-york,Sonali Indian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/0_OtiR...,False,https://www.yelp.com/biz/sonali-indian-restaur...,3,"[{'alias': 'indpak', 'title': 'Indian'}]",2.5,"{'latitude': 40.7266693115234, 'longitude': -7...",[],NaN,"{'address1': '326 E 6th St', 'address2': '', '...",+12125057517,(212) 505-7517,1133.773854
106,g8i7ZoTri1fUGkaIM6fP9A,souen-new-york-2,Souen,https://s3-media4.fl.yelpcdn.com/bphoto/X9Guht...,False,https://www.yelp.com/biz/souen-new-york-2?adju...,350,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 40.7266693115234, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '326 E 6th St', 'address2': '', '...",+12123881155,(212) 388-1155,1133.708444
107,rPqY0-3Ij3KMLEFvB730vw,parantha-alley-new-york-2,Parantha Alley,https://s3-media3.fl.yelpcdn.com/bphoto/T_Yv_w...,False,https://www.yelp.com/biz/parantha-alley-new-yo...,1,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",1.0,"{'latitude': 40.72664, 'longitude': -73.99202}",[],NaN,"{'address1': '348 Bowery', 'address2': None, '...",+12124750850,(212) 475-0850,1237.820868
108,pzVUUB9bXZMK8KvRLOoUWg,kiran-continental-indian-cuisine-new-york,Kiran Continental Indian Cuisine,,False,https://www.yelp.com/biz/kiran-continental-ind...,1,"[{'alias': 'indpak', 'title': 'Indian'}]",3.0,"{'latitude': 40.71463, 'longitude': -74.00752}","[delivery, pickup]",$$,"{'address1': '94 Chambers St', 'address2': '',...",+12127324893,(212) 732-4893,1868.279554
109,0jqs32xyAioVbz7BBzrO6A,chinese-dhaba-new-york,Chinese Dhaba,,False,https://www.yelp.com/biz/chinese-dhaba-new-yor...,3,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",2.0,"{'latitude': 40.71469, 'longitude': -74.00768}","[delivery, pickup]",NaN,"{'address1': '98 Chambers St', 'address2': Non...",+19292658135,(929) 265-8135,1887.562220


In [47]:
# check for duplicates 
final_df.duplicated(subset='id').sum()

1

In [48]:
# drop duplicated ids and confirm there is no more 
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0